### Model 1 Naiye Baeyes

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('pre_title_steaming.csv')
# df=pd.read_csv('pre_title.csv')

In [3]:
df.head(10)

,title,url_berita,sentimen
0,strategi sleman dorong ekonomi pakai batik int...,https://20.detik.com/blak-blakan/20231204-2312...,1
1,erick thohir ajak muda tak mager indonesia leb...,https://20.detik.com/demi-indonesia/20231027-2...,1
2,pan x jakcloth pesta anak nongkrong sambang ba...,https://20.detik.com/advertorial/20231109-2311...,1
3,iringiringan awal presiden joe biden tabrak mobil,https://20.detik.com/detikjabar/20231218-23121...,1
4,greysia polii ajak anak muda terus lang raih p...,https://20.detik.com/demi-indonesia/20231027-2...,1
5,kapolri bicara arti sumpah pemuda indonesia,https://20.detik.com/demi-indonesia/20231027-2...,1
6,temu habib rizieq cak imin bantah bahas dukung,https://20.detik.com/detikupdate/20231001-2310...,1
7,kades bandung mundur dukung amin anies salut,https://20.detik.com/detikupdate/20231001-2310...,1
8,pdip sebut ganjar mulia tani nelayan jadi pres...,https://20.detik.com/detikupdate/20231001-2310...,1
9,anies temu alumni itb bahas metode ilmiah ambi...,https://20.detik.com/detikupdate/20231001-2310...,1


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [5]:
# Ubah nilai sentimen
def map_sentiment(sentiment):
    if sentiment == 1:
        return 'positif'
    elif sentiment == 0:
        return 'netral'
    else:
        return 'negatif'

# df = pd.read_csv('pre_title_steaming.csv')
# df = pd.read_csv('pre_title.csv')
df['sentimen'] = df['sentimen'].map(map_sentiment)

In [6]:
# Membagi data menjadi fitur dan label
X = df['title']
y = df['sentimen']

In [7]:
X

0        strategi sleman dorong ekonomi pakai batik int...
1        erick thohir ajak muda tak mager indonesia leb...
2        pan x jakcloth pesta anak nongkrong sambang ba...
3        iringiringan awal presiden joe biden tabrak mobil
4        greysia polii ajak anak muda terus lang raih p...
                               ...                        
14995    ganjar duka hamka haq tinggal dunia beliau ora...
14996    ziarah makam syiah kuala aceh cak imin sambut ...
14997    gunromli bicara ganjar luka diri obat ade arma...
14998    baswalu dki panggil semua pihak kait gibran ba...
14999    bahas formatpanelis debat kpu rapat tutup bare...
Name: title, Length: 15000, dtype: object

In [8]:
y

0        positif
1        positif
2        positif
3        positif
4        positif
          ...   
14995     netral
14996     netral
14997     netral
14998     netral
14999     netral
Name: sentimen, Length: 15000, dtype: object

In [9]:
def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels,
    test_size=0.33,random_state=42)
    return train_X, test_X, train_Y, test_Y

norm_train_corpus, norm_test_corpus, train_labels, test_labels = prepare_datasets(X,y,test_data_proportion=0.3)

In [10]:
norm_train_corpus[0:10]

655               faktafakta ledak tungku smelter morowali
12044    dengar keluh pupuk langka gresik cak imin pada...
14844    bw puji misi daulat air anies sangat dasar sek...
13985    beforeafter jalan lampung tengah dulu bak kuba...
7974     331 rumah warga bogor banjir imbas hujan 20 an...
8612     derita panjang gaza gegara israel tiada henti ...
6883      geger bakteri shigella serang tentara israel apa
2199                      nomor urut 1 anies selalu hikmah
2960     bawaslu minta anggota tak gentar awas netralit...
4640                    who kaget rs indonesia gaza serang
Name: title, dtype: object

In [11]:
norm_test_corpus[0:10]

11499    deret anak presiden terjun politik gibran ahy ...
6475     ganjar komentar soal transisi energi begini re...
13167    yasonna aku tak tahu mana wamenkumham status s...
862      faktafakta tembak pria bekas korban tewas luka...
5970     duga orang bantu tahan lapas tangerang kabur a...
6706     dicurhati bbm langka lama kampanye ganjar haru...
3017     datang lokasi debat cawapres young lex gua bar...
3781     wamenlu bantu palestina bentuk solidaritas ind...
3898     doa palestina rizki sadig ajak warga jatim bac...
2250     depan mahasiswa stanford jokowi cerita ri puny...
Name: title, dtype: object

In [12]:
#Fungsi untuk mengekstraksi feature menggunakan TF-IDF Model
def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm='l2',
                                    smooth_idf=True,
                                    use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix
def tfidf_extractor(corpus, ngram_range=(1,1)):
    vectorizer = TfidfVectorizer(min_df=1,
                                norm='l2',
                                smooth_idf=True,
                                use_idf=True,
                                ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [13]:
#mengunakan model TF-IDF untuk mengekstraksi feature
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(norm_train_corpus)
tfidf_test_features = tfidf_vectorizer.transform(norm_test_corpus)

In [14]:
from sklearn.linear_model import SGDClassifier
train_features=tfidf_train_features
train_labels=train_labels
test_features=tfidf_test_features
test_labels=test_labels

In [15]:
df_train = pd.DataFrame()
df_train['norm_train'] = norm_train_corpus
df_train['train_labels'] = train_labels
df_test = pd.DataFrame()
df_test['norm_test'] = norm_test_corpus
df_test['test_labels'] = test_labels

In [16]:
df_train.head()

,norm_train,train_labels
655,faktafakta ledak tungku smelter morowali,positif
12044,dengar keluh pupuk langka gresik cak imin pada...,netral
14844,bw puji misi daulat air anies sangat dasar sek...,netral
13985,beforeafter jalan lampung tengah dulu bak kuba...,netral
7974,331 rumah warga bogor banjir imbas hujan 20 an...,negatif


In [17]:
df_test.head()

,norm_test,test_labels
11499,deret anak presiden terjun politik gibran ahy ...,netral
6475,ganjar komentar soal transisi energi begini re...,negatif
13167,yasonna aku tak tahu mana wamenkumham status s...,netral
862,faktafakta tembak pria bekas korban tewas luka...,positif
5970,duga orang bantu tahan lapas tangerang kabur a...,negatif


In [18]:
train_features[0:1]

<1x8933 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [19]:
train_labels[0:1]

655    positif
Name: sentimen, dtype: object

In [20]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

In [21]:
clf = SGDClassifier(loss='hinge', max_iter=100)
#clf = LinearSVC(penalty='l2', loss='squared_hinge', max_iter=100)
#clf = MultinomialNB()

In [22]:
clf.fit(train_features, train_labels)
predictions = clf.predict(test_features)
print("Label test: \n" + format(test_labels[:10]))
print("Prediction test: \n"+ format(predictions[:10]))

Label test: 
11499     netral
6475     negatif
13167     netral
862      positif
5970     negatif
6706     negatif
3017     positif
3781     positif
3898     positif
2250     positif
Name: sentimen, dtype: object
Prediction test: 
['netral' 'negatif' 'netral' 'positif' 'netral' 'netral' 'netral' 'netral'
 'positif' 'netral']


In [23]:
print("Accuration: "+format(clf.score(test_features,test_labels)))

Accuration: 0.6703030303030303


In [24]:
import numpy as np
from sklearn import metrics

In [25]:
def get_metrics(true_labels, predicted_labels):
    print('Accuracy: ', np.round(metrics.accuracy_score(true_labels,
                                                        predicted_labels),2))
    print('Precision: ', np.round(metrics.precision_score(true_labels,
                                                        predicted_labels,
                                                        average='weighted'),2))
    print('Recall: ', np.round(metrics.recall_score(true_labels,
                                                        predicted_labels,
                                                        average='weighted'),2))
    print('F1 Score: ', np.round(metrics.f1_score(true_labels,
                                                        predicted_labels,
                                                        average='weighted'),2))
# mengevaluasi performa prediksi menggunakan model
get_metrics(true_labels=test_labels,predicted_labels=predictions)

Accuracy:  0.67
Precision:  0.67
Recall:  0.67
F1 Score:  0.67


In [26]:
test = ('Jeka Saragih On Fire Menatap Debut di UFC')

In [27]:
line = tfidf_vectorizer.transform([test])
line.shape

(1, 8933)

In [28]:
line

<1x8933 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [29]:
test = clf.predict(line)

In [30]:
test

array(['negatif'], dtype='<U7')